In [ ]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import requests
from bs4 import BeautifulSoup

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
print("Hangi filtrelemeyi kullanmak istiyorsunuz?:\n")
print("Content Based Filtering Yöntemi için => 1")
print("Soru Sorarak Filtreleme Yöntemi için => 2")
print("Collaborative Filtering Yöntemi için => 3")

In [ ]:
userChoice = int (input("Filtreleme yöntemini giriniz:\n"))

In [ ]:
if (userChoice==1):
  
  #Store the data
  df = pd.read_csv('IMDB-Movie-Data.csv')
  #Show the firs 3 rows of data
  df.head(3)
  #Get a count of the number of rows/movies in the data set and the number of columns
  df.shape
  #Create a function to combine values of the important columns into a single string
  def get_important_features(data): 
    important_features = []
    for i in range(0, data.shape[0]):
      important_features.append(data['Actors'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Title'][i])

    return important_features

  #Create a column to hold the combined strings
  df['important_features'] = get_important_features(df)
  #Show the data
  df.head(3)
  #Convert the text to a matrix of token counts
  cm = CountVectorizer().fit_transform(df['important_features'])
  #Get the cosine similarity matrix from the count matrix
  cs = cosine_similarity(cm)
  #Print the cosine similarity matrix
  #print(cs)
  #Get the shape of the cosine similarity matrix 
  cs.shape
  df['Movie_id'] = range(0,1000)
  #Get the title of the movie that the user likes
  print("Orjinal Film ismini giriniz :\n")
  title = str(input()).capitalize()
  #Find the movies id
  movie_id = df[df.Title ==title]['Movie_id'].values[0]
  #Create a list of enumerations for the similarity score( movie_id,similarity score)
  scores = list(enumerate(cs[movie_id]))
  #Sort the list
  sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)
  sorted_scores = sorted_scores[1:]
  #Print the sorted scores
  #print(sorted_scores)
  #Create a loop to print the first 7 similar movies
  j = 0
  print('En Çok Önerilen 7 Film',title,'izleyenler için','\n')
  for item in sorted_scores:
    movie_title = df[df.Movie_id == item[0]]['Title'].values[0]
    print(j+1, movie_title)
    j = j+1
    if j>6:
      break

elif (userChoice==2):
  def get_movies(start, genres, release_year, imdb_score, runtime, certificates):
      # Set up the base URL
      url = "https://www.imdb.com/search/title"
      params = {
          "title_type": "feature,tv_movie",
          "count": "250",
          "start": start,
          "ref_": "adv_nxt",
      }
      # Add genres if specified
      if genres:
          for i, genre in enumerate(genres):
              params[f"genres{i}"] = genre
      # Add release year, IMDb score, runtime, and certificates if specified
      if release_year:
          params["release_date"] = f"{release_year},2022-11-01"
      if imdb_score:
          params["user_rating"] = f"{imdb_score},10.0"
      if runtime:
          params["runtime"] = f",{runtime},180"
      if certificates:
          params["certificates"] = certificates

      # Make the request and parse the response
      response = requests.get(url, params=params)
      soup = BeautifulSoup(response.text, "html.parser")

      # Extract the movie data
      movies = []

      movie_containers = soup.find_all("div", class_="lister-item mode-advanced")
      for container in movie_containers:
          # Extract the title and release year
          title_year_section = container.h3
          title = title_year_section.a.text
          release_year = title_year_section.find("span", class_="lister-item-year").text[1:5]
          # Extract the IMDb score
          imdb_rating_div = container.find("div", class_="inline-block ratings-imdb-rating")
          if imdb_rating_div:
              imdb_score = imdb_rating_div.strong.text
          else:
              imdb_score = ""
          # Extract the runtime
          if container.find("span", class_="runtime") is not None:
            runtime = container.find("span", class_="runtime").text[:-4]
          else:
            runtime = ""

          # Append the movie data as a tuple
          movies.append((title, release_year, imdb_score, runtime))
      return movies
  # Set up the list of genres
  all_genres = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family", "fantasy", "film noir", "history", "horror", "music", "musical",
            "mystery", "romance", "sci-fi", "short film", "sport", "superhero", "thriller", "war", "western"]

  # Get the user's input
  genre_number = int(input("Kaç farklı film türüne göre film filtrelemek istiyorsunuz?\n"))
  print("                                                                                                                             ")
  print("Film Türleri : ")
  if genre_number > 0:
      for i in all_genres:
          print(i);
       
      genre_indices = []
      print("                                                                                                                             ")
      print("Seçtiğiniz film türünün indexini giriniz :  ")
      for i in range(genre_number):
          genre_index = int(input())
          genre_indices.append(genre_index)
      genres = [all_genres[i] for i in genre_indices]
  else:
      genres = None
  
  release_year_input = input("Filmlerin çıkış yılı en az kaç olmalıdır? Yayın yılına göre filtrelememek için 0 girin.\n")
  release_year = release_year_input if release_year_input != "0" else None

  imdb_score_input = input("Filmlerin minimum IMDb puanı kaçtır? IMDb puanına göre filtrelememek için 0 girin.\n")
  imdb_score = imdb_score_input if imdb_score_input != "0" else None

  runtime_input = input("Filmlerin maksimum süresi dakika biçiminden kaç dakika olmalıdır? Filmlerin süresine göre filtrelememek için o girin.\n")
  runtime = runtime_input if runtime_input != "0" else None

  certificates = input("Hangi sertifikaları dahil etmek istiyorsunuz? Sertifikaların arasına , koyarak girin. Sertifikalara göre filtrelememek için none girin.\n")
  if certificates == "":
      certificates = None

  # Get the movie data
  movies = get_movies(0, genres, release_year, imdb_score, runtime, certificates)
  

  # Print the movie data
  for movie in movies:
      print(f"Title: {movie[0]}, Release Year: {movie[1]}, IMDb Score: {movie[2]}, Runtime: {movie[3]}")  

elif(userChoice==3):
  ratings = pd.read_csv('ratings.csv')
  movies = pd.read_csv('movies.csv')

  ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
  ratings.head()

  user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
  user_ratings.head()

  user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)
  user_ratings.head()

  item_similarity_df = user_ratings.corr(method='pearson')
  item_similarity_df.head(100)

  def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name] * (user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False) 
    return similar_score
  movies_array = []

  print("Film ismi formatı şu şekilde olmalıdır: filmin ismi (tarihi), filmin isminin baş harfi büyük olmalıdır")

# Prompt the user to input a movie title and rating
  while True:
    movie = input("Film ismi giriniz ya da çıkmak için 'q' tuşunu gir: ")
    if movie == 'q':
      break
    rating = input("Girdiğin filme 1 ile 5 arası puan verin: ")
    rating = float(rating)
  # Append the movie and rating to the list
    movies_array.append((movie,rating))

# Create an empty dataframe to store the recommendations
  similar_movies = pd.DataFrame()


  for movie,rating in movies_array:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)

# Sort the recommendations by the sum of their similarity scores in descending order
  recommendations = similar_movies.sum().sort_values(ascending=False)

# Print the recommendations
  print(recommendations)
  